In [80]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from bs4 import BeautifulSoup
from urllib.request import urlopen

Using beautiful soup to scrape wiki page

In [81]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html1 = urlopen(url) 
soup = BeautifulSoup(html1, 'html.parser')

Convert data into dataframe,Ignoring the Not Assigned Borough rows.

In [89]:
#Create array to hold the data we extract
table = soup.find_all('table',attrs={'class': 'wikitable sortable'})
df = pd.read_html(str(table))[0]
df = df[df.Borough != 'Not assigned']
df['Neighbourhood'] = np.where((df['Neighbourhood'] == 'Not assigned'),df['Borough'],df['Neighbourhood']) 
df.reset_index(drop=True, inplace=True)
print(df.head(10))
print("Data shape",df.shape)

  Postal Code           Borough                                Neighbourhood
0         M3A        North York                                    Parkwoods
1         M4A        North York                             Victoria Village
2         M5A  Downtown Toronto                    Regent Park, Harbourfront
3         M6A        North York             Lawrence Manor, Lawrence Heights
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government
5         M9A         Etobicoke      Islington Avenue, Humber Valley Village
6         M1B       Scarborough                               Malvern, Rouge
7         M3B        North York                                    Don Mills
8         M4B         East York              Parkview Hill, Woodbine Gardens
9         M5B  Downtown Toronto                     Garden District, Ryerson
Data shape (103, 3)
